In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2024  # Can increase for longer reasoning traces
lora_rank = 32  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="gsarti/phi3-mini-rebus-solver-fp16",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=False,  # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6,  # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Remove QKVO if out of memory
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",  # Enable long context finetuning
    random_state=3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-12 14:46:52.167721: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-12 14:46:52.852592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-12 14:46:52.852623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-12 14:46:52.871215: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-12 14:46:52.913412: I tensorflow/core/platform/cpu_feature_guar

Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [16]:
import re

def estrai_soluzione(input_string):
    # Extract only the solution
    match = re.search(r"Soluzione: (.+?)\n", input_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return "NotFound"

def estrai_indizi(input_string):

    # Extract the `[...] = relevantPart`
    pattern = r"\[([^\]]+)\] = ([^\n]+)"
    indizi = re.findall(pattern, input_string)
    risposte = [risposta for _, risposta in indizi]
    
    # Extract the `... = relevantLetters`
    pattern_sigle = r"[-–•]?\s*([A-Z]+(?:\s+[A-Z]+)*)\s*=\s*[^\n]+"    
    risposte_sigle = re.findall(pattern_sigle, input_string)
    
    # Combina le risposte estratte dalle parentesi e quelle singole
    return {'letters': risposte_sigle, 'words': risposte}

def estrai_primalet(input_string):
    # Extract the first-pass (prima lettura in italian)
    match = re.search(r"Prima lettura: (.+?)\n", input_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return "NotFound"


def estrai_rebus_e_chiave(testo):
    # Extract from the problem formulation: 
    #   - the rebus problem 
    #   - the key (i.e. bounds of number of letters of the solution)
    rebus_match = re.search(r"Rebus:\s*(.+?)\s*Chiave di lettura:", testo, re.DOTALL)
    chiave_match = re.search(r"Chiave di lettura:\s*(.+)", testo)

    rebus_raw = rebus_match.group(1).strip() if rebus_match else ""
    chiave = chiave_match.group(1).strip() if chiave_match else ""

    return rebus_raw, chiave

In [17]:
def exact_match_solution(prompts, completions, ground_truth, **kwargs) -> list[float]:
    # Estrazione delle soluzioni
    predicted = [estrai_soluzione(completion) for completion in completions]
    gold = estrai_soluzione(ground_truth[0])
    print(predicted)
    print (gold)
    
    scores = []
    for guess in predicted:
        if guess == "NotFound":
            scores.append(0)
            continue
        try:
            scores.append(1.0 if guess == gold else 0.0)
        except:
            scores.append(0)
            continue
    return scores


def perc_correct_words_solution(prompts, completions, ground_truth, **kwargs):
    gold = estrai_soluzione(ground_truth[0]).lower().split()
    scores = []
    
    for completion in completions:
        print(completion)
        pred = estrai_soluzione(completion)
        print(pred)
        if not pred:
            continue

        pred = pred.lower().split()
        score = 0
        for pw, gw in zip(pred, gold):
            if pw == gw:
                score += 1
            elif len(pw) == len(gw):
                score += 0.5
        scores.append(score / len(gold))

    return scores


def exact_match_primalet(prompts, completions, ground_truth, **kwargs):
    predicted = [estrai_primalet(completion) for completion in completions]
    golden = estrai_primalet(ground_truth[0]).lower().replace(" ", "")
    scores = []
    for guess in predicted:
        if guess == "NotFound":
            scores.append(0)
            continue
        try:
            scores.append(1.0 if guess.lower().replace(" ", "") == golden else 0.0)
        except:
            scores.append(0)
            continue
    return scores


def perc_correct_defres(prompts, completions, ground_truth, **kwargs):
    predicted = [estrai_indizi(completion.replace("*", "")) for completion in completions] 
    golden = estrai_indizi(ground_truth[0])
    word_scores = []
    letter_scores = []
    for pred in predicted:
        wscore = 0
        for pw, gw in zip(pred['words'], golden['words']):
            if pw == gw:
                wscore += 1
            elif len(pw) == len(gw):
                wscore += 0.5
        word_scores.append(wscore / len(golden['words']))

        lscore = 0
        for pw, gw in zip(pred['letters'], golden['letters']):
            if pw.lower().replace(" ", "") == gw.lower().replace(" ", ""):
                lscore += 1
        letter_scores.append(lscore / len(golden['letters']))
        
    return [word_scores[i] + letter_scores[i] for i in range(len(predicted))]

In [27]:
from datasets import load_dataset
dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["train.jsonl"], split = "train")

In [28]:
import pandas as pd

In [29]:
def formatting_prompts_func(examples, model_name):
    
    if model_name == "gsarti/phi3-mini-rebus-solver-fp16":
        template = """<s><|user|>
        Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.
        
        Rebus: {rebus}
        Chiave risolutiva: {key}<|end|>
        <|assistant|>"""
        
    elif model_name == "gsarti/llama-3.1-8b-rebus-solver-fp16":
        template = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
        
        Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.
        
        Rebus: {rebus}
        Chiave risolutiva: {key}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
    prompt_list = []
    completion_list = []
    
    for i in range(len(dataset)):
        # Estrai rebus e chiave
        rebus_raw, chiave = estrai_rebus_e_chiave(dataset[i]['conversations'][0]['value'])
        
        # Crea il prompt usando il template
        prompt = template.format(rebus=rebus_raw, key=chiave)
        
        # Ottieni la completion dal dataset
        completion = dataset[i]['conversations'][1]['value']
        
        # Aggiungi a lista
        prompt_list.append(prompt)
        completion_list.append(completion)
    
    # Crea il DataFrame
    df = pd.DataFrame({
        'prompt': prompt_list,
        'completion': completion_list
    })

    return df

In [36]:
input = formatting_prompts_func(dataset, model_name="gsarti/phi3-mini-rebus-solver-fp16")

In [37]:
input.head()

,prompt,completion
0,<s><|user|>\n Risolvi gli indizi tra pa...,Procediamo alla risoluzione del rebus passo pe...
1,<s><|user|>\n Risolvi gli indizi tra pa...,Procediamo alla risoluzione del rebus passo pe...
2,<s><|user|>\n Risolvi gli indizi tra pa...,Procediamo alla risoluzione del rebus passo pe...
3,<s><|user|>\n Risolvi gli indizi tra pa...,Procediamo alla risoluzione del rebus passo pe...
4,<s><|user|>\n Risolvi gli indizi tra pa...,Procediamo alla risoluzione del rebus passo pe...


In [47]:
inputs = tokenizer(input['prompt'][0], return_tensors="pt")["input_ids"].to('cuda:0')
outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, do_sample=True)
model_generations = tokenizer.batch_decode(outputs)

In [48]:
model_generations

['<s><s><|user|> Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.\n        \n        Rebus: A [Quello di fiori si riceve volentieri] [Vezzi visibili] RL [Dividono l\'Argentina dal Cile] SE\n        Chiave risolutiva: 8 9<|end|><|assistant|> Per risolvere il rebus seguendo gli indizi e la chiave di lettura, possiamo tradurre gli indizi letteralmente:\n\n1. [Quello di fiori si riceve volentieri] = Fiori (ma dato che il risultato deve essere un numero, non puo\' significare letteralmente fiori)\n2. [Vezzi visibili] = Biche (dipendente dal contesto, ma un "biche" o tronco arrotondato visibile potrebbe funzionare come indizio)\n3. [Dividono l\'Argentina dal Cile] = Meridiani (come il meridiano è uno dei divisori della Terra, ma non risolve il rebus)\n4. SE è un codice (SE=4 nel caso del codice dei simboli di Freud e non interessa al rebus)\n\nPrendendo in considerazione anche la chiave risolutiva (8 9), il puzzl

In [49]:
parse_generation(1, model_generations[0])

{'idx': 1,
 'word_guesses': '',
 'first_pass': '',
 'solution_words': '',
 'solution': ''}

In [1]:
import re 

regex_word_guess = '- \[.* = (.*)'
regex_firstpass = 'Prima lettura: (.*)'
regex_solution_word = "\d+ = (.*)"
regex_solution = "Soluzione: (.*)"

def parse_generation(ex_idx, ex):
    try:
        word_guesses = ";".join(re.findall(regex_word_guess, ex))
    except:
        word_guesses = ""
    try:
        first_pass = re.findall(regex_firstpass, ex)[0]
    except:
        first_pass = ""
    try:
        solution_words = ";".join(re.findall(regex_solution_word, ex))
    except:
        solution_words = ""
    try:
        solution = re.findall(regex_solution, ex)[0]
    except:
        solution = ""
    return {
        "idx": ex_idx,
        "word_guesses": word_guesses,
        "first_pass": first_pass,
        "solution_words": solution_words,
        "solution": solution,
    }

In [2]:
eval_dataset = [
    {"conversations": [{"value": "Indovina la parola misteriosa!"}]},
    {"conversations": [{"value": "Qual è la soluzione per il problema?"}]},
    {"conversations": [{"value": "Risolvere il puzzle è difficile."}]}
]


In [50]:
completions = []

for i in range(4):
    outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, do_sample=True)
    model_generations = tokenizer.batch_decode(outputs)
    completions.append(model_generations[0])

In [54]:
# completions

In [51]:
# Eseguiamo la funzione parse_generation esplicitamente per ogni esempio
results = []

for ex_idx, ex in enumerate(completions):
    # Chiamata esplicita della funzione parse_generation
    parsed_result = parse_generation(ex_idx, ex)
    
    # Stampa il risultato per ogni esempio
    print(f"Risultato per esempio {ex_idx + 1}:")
    print(parsed_result)
    
    # Aggiungi il risultato alla lista results
    results.append(parsed_result)

Risultato per esempio 1:
{'idx': 0, 'word_guesses': '', 'first_pass': '', 'solution_words': '', 'solution': ''}
Risultato per esempio 2:
{'idx': 1, 'word_guesses': '', 'first_pass': '', 'solution_words': '', 'solution': ''}
Risultato per esempio 3:
{'idx': 2, 'word_guesses': '', 'first_pass': '', 'solution_words': '', 'solution': ''}
Risultato per esempio 4:
{'idx': 3, 'word_guesses': '', 'first_pass': '', 'solution_words': '', 'solution': ''}


In [ ]:
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=6,  # Decrease if out of memory
    max_prompt_length=max_prompt_length,
    max_completion_length=500,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[exact_match_solution, perc_correct_words_solution,
                  exact_match_primalet, perc_correct_defres],
    args=training_args,
    train_dataset=dataset,
)

In [ ]:
# trainer.train()